In [278]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

**Get Data**

In [226]:
test_df = pd.read_csv('../input/test.csv')
train_df = pd.read_csv('../input/train.csv')
combine = [train_df, test_df]
train_df.shape, test_df.shape

In [227]:
test_df.head()

In [228]:
train_df.describe()

In [229]:
train_df.columns.values

In [230]:
train_df.isnull().any()

In [231]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

In [232]:
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()

**Clean Data**

In [233]:
train_df = train_df.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin', 'Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

**Prepare Data**

In [234]:
sex_mapping = {"male": 0, "female": 1}
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)
    dataset['Sex'] = dataset['Sex'].fillna(0)

train_df.head()

In [235]:
for dataset in combine:
    dataset['Age'] = dataset['Age'].replace(np.nan, dataset['Age'].mean(), regex=True)
    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

In [236]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [237]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
    
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

In [238]:
frequence_port = train_df.Embarked.dropna().mode()[0]

'''
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].replace(np.nan, "M", regex=True)
'''
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(frequence_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean()

In [239]:
#embark_mapping = {"S": 0, "C": 1, "Q": 2, "M": 3}

embark_mapping = {"S": 0, "C": 1, "Q": 2}
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map(embark_mapping).astype(int)

train_df.head()

In [240]:
test_df.isnull().any()

In [241]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

In [242]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean()

In [243]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

In [244]:
test_df.head(10)

In [245]:
train_df.head(10)

In [246]:
predict_train = train_df.drop("Survived", axis=1)
predict_test = test_df.drop("PassengerId", axis=1)
X_train = predict_train
Y_train = train_df["Survived"]
X_test  = predict_test
X_train.shape, Y_train.shape, X_test.shape

**Train Model**

In [287]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_dct_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [288]:
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
Y_knn_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

**Test Data**

In [289]:
submission = pd.read_csv('../input/gender_submission.csv')
submission[0:20]

In [290]:
output_dct = pd.DataFrame({"PassengerId": test_df["PassengerId"], "Survived": Y_dct_pred})
output_dct[0:20]

**Improve**